<a href="https://colab.research.google.com/github/ju-ki/pytorch_basic/blob/master/autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* pytorchには自動微分というモジュールがある。

In [1]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


### pytorchを使用した簡単な順伝播と逆伝播

In [2]:
#forward pass
prediction = model(data)

In [4]:
loss = (prediction - labels).sum()
#backward pass
loss.backward()

In [5]:
#最適化の手法を選択
optim = torch.optim.SGD(model.parameters(), lr=1e-6, momentum=0.9)

In [6]:
#勾配
optim.step()

### 自動微分

In [7]:
#requires_grad=Trueとすることで計算過程を見ることができる。
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
Q = 3*a**3 - b**2

In [8]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)#逆伝播により勾配を算出

In [9]:
#手動でやった場合と自動微分を行った際のデバッグ
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


In [10]:
#一つでもTrueになっていれば足し算しても反映される
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does 'a' require gradients?: {a.requires_grad}")
b = x + z
print(f"Does 'b' require gradients?: {b.requires_grad}")

Does 'a' require gradients?: False
Does 'b' require gradients?: True


In [11]:
from torch import nn, optim #ニューラルネットワークと最適化アルゴリズムのインポート

model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
  param.requires_grad = False

In [12]:
model.fc = nn.Linear(512, 10)

In [13]:
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)

In [14]:
#(2, 3)のテンソルを定義
x = torch.ones(2, 3, requires_grad=True)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)


In [15]:
#grad_fnという所にADDBackwardと出力される
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3., 3.],
        [3., 3., 3.]], grad_fn=<AddBackward0>)


In [19]:
#掛け算
z = x * 3
print(z)
#割り算
q = x / 3
print(q)
#平均
out = z.mean()
print(out)
#Max
max = z.max()
print(max)
#中央値
median = z.median()
print(median)

tensor([[3., 3., 3.],
        [3., 3., 3.]], grad_fn=<MulBackward0>)
tensor([[0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333]], grad_fn=<DivBackward0>)
tensor(3., grad_fn=<MeanBackward0>)
tensor(3., grad_fn=<MaxBackward1>)
tensor(3., grad_fn=<MedianBackward0>)


### 勾配の算出

In [20]:
a = torch.tensor([1.0], requires_grad=True)
b = a * 2
b.backward()
print(a.grad)#aの変化量に対するbの変化の割合

tensor([2.])
